In [2]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from iller import iller

# Ay ve gün bilgileri
aylar = {
    'OCAK': 1, 'SUBAT': 2, 'MART': 3, 'NISAN': 4, 'MAYIS': 5, 'HAZIRAN': 6,
    'TEMMUZ': 7, 'AGUSTOS': 8, 'EYLUL': 9, 'EKIM': 10, 'KASIM': 11, 'ARALIK': 12
}
gun_sayisi = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

# Tahmin hesaplama fonksiyonu
def hesapla_tahminler(ay1, ay2, gun_fark, baslangic_tarihi):
    ust_fark = ay2['UST_SICAKLIK'] - ay1['UST_SICAKLIK']
    alt_fark = ay2['ALT_SICAKLIK'] - ay1['ALT_SICAKLIK']
    ust_gunluk_fark = ust_fark / gun_fark
    alt_gunluk_fark = alt_fark / gun_fark

    tahminler = []
    for g in range(gun_fark):
        tarih = baslangic_tarihi + timedelta(days=g)
        gun_ust_tahmin = round(ay1['UST_SICAKLIK'] + ust_gunluk_fark * g, 2)
        gun_alt_tahmin = round(ay1['ALT_SICAKLIK'] + alt_gunluk_fark * g, 2)
        tahminler.append([tarih.strftime('%Y-%m-%d'), gun_ust_tahmin, gun_alt_tahmin])
    return tahminler

# MySQL bağlantı bilgileri
mysql_config = {
    'user': 'root',
    'password': 'Luxilionalupower125',
    'host': 'localhost',
    'port': 3306,
    'database': 'ecobloom'  # ← burayı kendi DB adına göre değiştir
}

conn = None
cursor = None

try:
    conn = mysql.connector.connect(**mysql_config)
    cursor = conn.cursor()

    for sehir_id, il_data in enumerate(iller, 1):
        df = pd.DataFrame({
            'KONUM': il_data['KONUM'],
            'AY': il_data['AY'],
            'UST_SICAKLIK': il_data['UST_SICAKLIK'],
            'ALT_SICAKLIK': il_data['ALT_SICAKLIK']
        })
        df['AY_NUMARASI'] = df['AY'].map(aylar)

        tahminler = []
        baslangic_tarihi = datetime(2026, 1, 1)

        for i in range(len(df) - 1):
            ay1, ay2 = df.iloc[i], df.iloc[i + 1]
            gun_fark = gun_sayisi[ay2['AY_NUMARASI']]
            tahminler.extend(hesapla_tahminler(ay1, ay2, gun_fark, baslangic_tarihi))
            baslangic_tarihi += timedelta(days=gun_fark)

        # Aralık - Ocak bağlantısı
        aralik_ay = df[df['AY'] == 'ARALIK'].iloc[0]
        ocak_ay = df[df['AY'] == 'OCAK'].iloc[0]
        gun_fark_aralik_ocak = gun_sayisi[ocak_ay['AY_NUMARASI']]
        tahminler.extend(hesapla_tahminler(aralik_ay, ocak_ay, gun_fark_aralik_ocak, baslangic_tarihi))

        tahmin_df = pd.DataFrame(tahminler, columns=['Tarih', 'TAHMIN_UST_SICAKLIK', 'TAHMIN_ALT_SICAKLIK'])

        insert_query = """
        INSERT INTO SEHIR_HAVA_DURUMU (TARIH, TAHMIN_ALT_SICAKLIK, TAHMIN_UST_SICAKLIK, SEHIR_ID)
        VALUES (%s, %s, %s, %s)
        """

        for _, row in tahmin_df.iterrows():
            cursor.execute(insert_query, (row['Tarih'], row['TAHMIN_ALT_SICAKLIK'], row['TAHMIN_UST_SICAKLIK'], sehir_id))

        conn.commit()
        print(f"{il_data['KONUM'][0]} verileri başarıyla MySQL'e kaydedildi.")

except mysql.connector.Error as err:
    print("MySQL Hatası:", err)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


ADANA verileri başarıyla MySQL'e kaydedildi.
ADIYAMAN verileri başarıyla MySQL'e kaydedildi.
AFYONKARAHISAR verileri başarıyla MySQL'e kaydedildi.
AGRI verileri başarıyla MySQL'e kaydedildi.
AMASYA verileri başarıyla MySQL'e kaydedildi.
ANKARA verileri başarıyla MySQL'e kaydedildi.
ANTALYA verileri başarıyla MySQL'e kaydedildi.
ARTVIN verileri başarıyla MySQL'e kaydedildi.
AYDIN verileri başarıyla MySQL'e kaydedildi.
BALIKESIR verileri başarıyla MySQL'e kaydedildi.
BILECIK verileri başarıyla MySQL'e kaydedildi.
BINGOL verileri başarıyla MySQL'e kaydedildi.
BITLIS verileri başarıyla MySQL'e kaydedildi.
BOLU verileri başarıyla MySQL'e kaydedildi.
BURDUR verileri başarıyla MySQL'e kaydedildi.
BURSA verileri başarıyla MySQL'e kaydedildi.
CANAKKALE verileri başarıyla MySQL'e kaydedildi.
CANKIRI verileri başarıyla MySQL'e kaydedildi.
CORUM verileri başarıyla MySQL'e kaydedildi.
DENIZLI verileri başarıyla MySQL'e kaydedildi.
DIYARBAKIR verileri başarıyla MySQL'e kaydedildi.
EDIRNE verileri ba